In [4]:
import os

In [5]:
os.chdir('../')

In [6]:
%pwd

'd:\\ML Block\\Projects\\ETE AI Projects\\kidney-disease-classification-dl-project'

In [7]:
from dataclasses import dataclass
from pathlib import Path


In [8]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [9]:
from src.kidney_cnn_classifier.constants import *
from src.kidney_cnn_classifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"kidney_ct_scan_image_normal_tumor")
        create_directories([
            Path(training.root_dir)
        ])


        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [17]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
    
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size


        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [18]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2024-11-22 01:08:12,852: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-22 01:08:12,854: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-22 01:08:12,855: INFO: common: created directory at: artifacts]
[2024-11-22 01:08:12,856: INFO: common: created directory at: artifacts\training]
Found 1471 images belonging to 2 classes.
Found 5889 images belonging to 2 classes.
368/368 [==============================] - 427s 1s/step - loss: 10.1067 - accuracy: 0.6278 - val_loss: 3.8928 - val_accuracy: 0.7246


c:\Users\vamsi7\anaconda3\envs\prj_kdny\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
import tensorflow as tf
print(tf.__version__)



2.13.0


In [20]:
import tensorflow as tf
print(tf.__version__)
print("CUDA Available:", tf.test.is_gpu_available())


2.13.0
[2024-11-22 01:29:36,375: WARNING: deprecation: From C:\Users\vamsi7\AppData\Local\Temp\ipykernel_45936\4095760562.py:3: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.]
CUDA Available: False


In [3]:
import PIL